# Shortest path algorithms
Shortest path is the core of operations research.
The shortest path algorithms can be devided into many categories, such as:
1. Single objective VS multiobjective
2. Deterministic VS stochastic
3. Addtive VS nonadditive
4. Nonconstrained VS constrained
5. Static VS time-dependent\
...

I will try my best to cover all the categories and include some interesting variants.
The structure of this document will be:
1. Mathematical model.
2. Data structure.
3. Algorihtm.
4. Data.
5. Implementation.

## Single objective, deterministic, additive shortest path
### Model

<img src='images\SP_model.jpg' width='600'>

In [5]:
# Data Stucture
class Node:
    """
    This class has attributes associated with any node
    """
    def __init__(self, Id):
        self.Id = Id
        self.outLinks = []
        self.inLinks = []
        '''Algorithm'''
        self.label = float("inf")
        self.pred = "" # predecessor

class Link:
    """
    This class has attributes associated with any link
    """
    def __init__(self, tail, head, cost):
        self.tail = tail
        self.head = head
        self.cost = cost

In [19]:
'Algorithm'
import heapq
def Dijkstra(nodeSet, linkSet, ori, des=None):
    '''
    Method: Dijkstra algorithm + binary heap
    Input: ori, des(optional)
    Return:
        if des != None, output the path nodes(list) and the dist(float);
        if des == None, output a path tree(dictionary of list) and the distances(dictionary of float), where key is the des.
            e.g., path, dist = Dijkstra(1), then path[2] and dist[2] denote the path node and distance from node 1 to node 2.
    '''
    # initialization
    for n in nodeSet:
        nodeSet[n].label, nodeSet[n].pred = (float("inf"), 'NA')
    nodeSet[ori].label, pred = (0.0, 'NA')
    # scan eligible list
    SE = [(0, ori)]
    # main loop
    while SE:
        currentNode = heapq.heappop(SE)[1]
        currentLabel = nodeSet[currentNode].label
        for l in nodeSet[currentNode].outLinks:
            nextNode = l[1]
            existingLabel = nodeSet[nextNode].label
            newLabel = currentLabel + linkSet[l].cost
            if newLabel < existingLabel:
                # add new label to scan eligible list
                heapq.heappush(SE, (newLabel, nextNode))
                # update label
                nodeSet[nextNode].label, nodeSet[nextNode].pred = (newLabel, currentNode)

    if des != None:
        path = tracePreds(des)
        dist = nodeSet[des].label
    else:
        path = {}
        dist = {}
        for n in nodeSet:
            path[n] = tracePreds(n)
            dist[n] = nodeSet[n].label

    return path, dist

def tracePreds(des):
    '''
    This method traverses predecessor nodes in order to create a path.
    '''
    pathNodes = [des]
    prevNode = nodeSet[des].pred
    while nodeSet[des].pred != "NA":
        pathNodes.append(prevNode)
        des = prevNode
        prevNode = nodeSet[des].pred
    list.reverse(pathNodes)

    return pathNodes

### Data 
#### From textbook: Ehrgott, M. (2005). Multicriteria optimization (Vol. 491). Springer Science & Business Media.
<img src="images/Textbook_network.jpg" width="800">
Use the first number at each link as the link cost.

In [20]:
'Data'
# tail, head, cost
linkData = [
    [1, 2, 10],
    [1, 3, 6],
    [2, 4, 0],
    [3, 2, 6],
    [3, 5, 1],
    [4, 3, 4],
    [4, 6, 10],
    [5, 4, 5],
    [5, 6, 6],
]

In [28]:
'Read data'
nodeSet = {}
linkSet = {}

for l in linkData:
    tail, head, cost = l
    # link
    linkSet[tail, head] = Link(tail, head, cost)
    # node
    if tail not in nodeSet:
        nodeSet[tail] = Node(tail)
    if head not in nodeSet:
        nodeSet[head] = Node(head)
    if (tail, head) not in nodeSet[tail].outLinks:
        nodeSet[tail].outLinks.append((tail, head))
    if (tail, head) not in nodeSet[head].inLinks:
        nodeSet[head].inLinks.append((tail, head))

print(len(nodeSet), "nodes")
print(len(linkSet), "links")

'Implementation'
Ori = 1
Des = 6
path, dist = Dijkstra(nodeSet, linkSet, Ori, Des)
print('Shortest path', path)
print('Cost', dist)

6 nodes
9 links
Shortest path {1: [1], 2: [1, 2], 3: [1, 3], 4: [1, 2, 4], 5: [1, 3, 5], 6: [1, 3, 5, 6]}
Cost {1: 0.0, 2: 10.0, 3: 6.0, 4: 10.0, 5: 7.0, 6: 13.0}


### What about the performance in big networks?
#### Chicago Sketch network: 933 nodes, 2950 links
<img src="images/Chicago_Sketch.png" width="600">

In [31]:
# Chicago Sketch network

import time

netFile = 'network\\ChicagoSketch.csv'

nodeSet = {}
linkSet = {}

inFile = open(netFile, 'r')
next(inFile)  # skip the first title line
for line in inFile:
    # data
    tmpIn = line.strip('\n').split(',')
    tail = int(tmpIn[0])
    head = int(tmpIn[1])
    cost = float(tmpIn[2])
    # link
    linkSet[tail, head] = Link(tail, head, cost)
    # node
    if tail not in nodeSet:
        nodeSet[tail] = Node(tail)
    if head not in nodeSet:
        nodeSet[head] = Node(head)
    if (tail, head) not in nodeSet[tail].outLinks:
        nodeSet[tail].outLinks.append((tail, head))
    if (tail, head) not in nodeSet[head].inLinks:
        nodeSet[head].inLinks.append((tail, head))

print(len(nodeSet), "nodes")
print(len(linkSet), "links")

'Implementation'
Ori = 1
Des = 600

tic = time.time()
path, dist = Dijkstra(nodeSet, linkSet, Ori, Des)
print('Running time=', time.time()-tic, 'sec.')

print('Shortest path', path)
print('Cost', dist)

933 nodes
2950 links
Running time= 0.001531839370727539 sec.
Shortest path [1, 547, 548, 552, 435, 554, 437, 438, 536, 537, 399, 398, 397, 396, 395, 600]
Cost 37.15


#### Sydney network: 33113 nodes, 75379 links
<img src="images/Sydney.jpg" width="600">

In [27]:
# Sydney network

import time

netFile = 'network\\Sydney.csv'

nodeSet = {}
linkSet = {}

inFile = open(netFile, 'r')
next(inFile)  # skip the first title line
for line in inFile:
    # data
    tmpIn = line.strip('\n').split(',')
    tail = int(tmpIn[0])
    head = int(tmpIn[1])
    cost = float(tmpIn[2])
    # link
    linkSet[tail, head] = Link(tail, head, cost)
    # node
    if tail not in nodeSet:
        nodeSet[tail] = Node(tail)
    if head not in nodeSet:
        nodeSet[head] = Node(head)
    if (tail, head) not in nodeSet[tail].outLinks:
        nodeSet[tail].outLinks.append((tail, head))
    if (tail, head) not in nodeSet[head].inLinks:
        nodeSet[head].inLinks.append((tail, head))

print(len(nodeSet), "nodes")
print(len(linkSet), "links")

'Implementation'
Ori = 1
Des = 6000

tic = time.time()
path, dist = Dijkstra(nodeSet, linkSet, Ori, Des)
print('Running time=', time.time()-tic, 'sec.')

print('Shortest path', path)
print('Cost', dist)

33113 nodes
75379 links
Running time= 0.07380223274230957 sec.
Shortest path [1, 6706, 30965, 6705, 30966, 6710, 6711, 30106, 6716, 82, 6691, 29723, 29218, 30091, 30090, 6698, 30092, 29224, 29225, 30104, 6709, 6695, 8899, 6641, 6642, 6522, 33100, 6482, 6371, 6359, 6348, 6333, 6256, 16493, 6233, 7944, 6182, 6059, 6038, 6000]
Cost 12.5


### The classical shortest path problem can be solved very quickly. There are also other speed-up strategies, including but not limited to adopting Fibonacci heap, bucket operation, heuristic algorithm (e.g. A*)...